<a href="https://colab.research.google.com/github/szhou52/HS-651/blob/main/30_day_readmission_Bert_500_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark==3.1.3 spark-nlp==3.4.2

     |████████████████████████████████| 214.0 MB 7.4 kB/s 
     |████████████████████████████████| 142 kB 82.5 MB/s 
     |████████████████████████████████| 198 kB 95.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.3-py2.py3-none-any.whl size=214463484 sha256=e6518d6211d605c2d30cd7f7adcc3647d09377a6913177258a9a263e460ac70b
  Stored in directory: /root/.cache/pip/wheels/ad/8e/49/44c110bb8e008d0778c6577d600d46047c6478ecca3f8f1517
Successfully built pyspark


In [ ]:
import sparknlp

spark = sparknlp.start(gpu=True)

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

spark

Spark NLP version:  3.4.2
Apache Spark version:  3.1.3


In [ ]:
import sys

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

In [ ]:
from pyspark.ml.feature import SQLTransformer
from pyspark.ml.feature import StringIndexer

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
readmission=pd.read_csv('gdrive/MyDrive/Colab_notebook/df_adm_dis_sum.csv')

Slice eath document into parts with 500 (or less) tokens each

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_test=train_test_split(readmission,test_size=0.2, random_state=42)

# sub-sampling the negatives (non-readmitted) on the training set
df_train_readm=df_train[df_train.READMISSION_STATUS=='Readmitted']
df_train_non_readm=df_train[df_train.READMISSION_STATUS=='Non-readmitted']
df_train_sub = pd.concat([df_train_readm, df_train_non_readm.sample(n = len(df_train_readm), random_state = 42)],axis = 0)

In [ ]:
df_train_sub=df_train_sub.reset_index(drop=True)
df_test=df_test.reset_index(drop=True)

In [ ]:
import math
def five_hund_split(text):
  sublist_list=[]
  token_list=text.split(" ")
  if len(token_list)<=500:
    sublist_list.append(text)
  else:
      num_sublist=int(math.modf(len(token_list)/500)[1]+1)
      sublist_list=[0]*num_sublist
      for i in range(num_sublist):
          sublist_list[i]=token_list[500*i:500*(i+1)]
          sublist_list[i]=" ".join(sublist_list[i])
  return(sublist_list)

In [ ]:
TEXT_AGG_five_hund=[]
HADM_ID=[]
READMISSION=[]

from tqdm import tqdm
for i in tqdm(range(len(df_train_sub))):
  sublist_list=five_hund_split(df_train_sub['TEXT_AGG'][i])
  admission_id=df_train_sub['HADM_ID'][i]
  readmission_status=df_train_sub['READMISSION_STATUS'][i]
  for sublist in sublist_list:
    TEXT_AGG_five_hund.append(sublist)
    HADM_ID.append(admission_id)
    READMISSION.append(readmission_status)

100%|██████████| 4522/4522 [00:00<00:00, 12402.95it/s]


In [ ]:
df_train_sliced= pd.DataFrame(list(zip(HADM_ID,TEXT_AGG_five_hund,READMISSION)),
               columns =['HADM_ID','TEXT_AGG','readmission_status'])

In [ ]:
TEXT_AGG_five_hund=[]
HADM_ID=[]
READMISSION=[]

from tqdm import tqdm
for i in tqdm(range(len(df_test))):
  sublist_list=five_hund_split(df_test['TEXT_AGG'][i])
  admission_id=df_test['HADM_ID'][i]
  readmission_status=df_test['READMISSION_STATUS'][i]
  for sublist in sublist_list:
    TEXT_AGG_five_hund.append(sublist)
    HADM_ID.append(admission_id)
    READMISSION.append(readmission_status)

100%|██████████| 9262/9262 [00:00<00:00, 12965.40it/s]


In [ ]:
df_test_sliced= pd.DataFrame(list(zip(HADM_ID,TEXT_AGG_five_hund,READMISSION)),
               columns =['HADM_ID','TEXT_AGG','readmission_status'])

In [ ]:
# Convert the pandas df to a spark df
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
train = spark.createDataFrame(df_train_sliced)
test= spark.createDataFrame(df_test_sliced)

Pipelines

In [ ]:
%%time
# Produce pipeline for data cleaning and sentence(discharge summary) embedding
document_assembler = DocumentAssembler() \
      .setInputCol("TEXT_AGG") \
      .setOutputCol("document")
    
tokenizer = Tokenizer() \
      .setInputCols(["document"]) \
      .setOutputCol("token")

lemmatizer = Lemmatizer() \
    .setInputCols(["token"]) \
    .setOutputCol("lemma") \
    .setDictionary("gdrive/MyDrive/Colab_notebook/AntBNC_lemmas_ver_001.txt", value_delimiter ="\t", key_delimiter = "->")

bert_embeddings = BertEmbeddings.pretrained()\
  .setInputCols(["document","lemma"])\
  .setOutputCol("bert_embeddings")\
  .setCaseSensitive(False)

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "bert_embeddings"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")

embeddings_finisher = EmbeddingsFinisher() \
      .setInputCols(["sentence_embeddings"]) \
      .setOutputCols(["finished_sentence_embeddings"]) \
      .setOutputAsVector(True)\
      .setCleanAnnotations(False)

explodeVectors = SQLTransformer(statement=
      "SELECT EXPLODE(finished_sentence_embeddings) AS features, * FROM __THIS__")

label_stringIdx = StringIndexer(inputCol = "readmission_status", outputCol = "label")

nlp_pipeline_Bert = Pipeline(
stages=[document_assembler, 
          tokenizer,
          lemmatizer,
          bert_embeddings,
          embeddingsSentence,
          embeddings_finisher,
          explodeVectors,
          label_stringIdx])

small_bert_L2_768 download started this may take some time.
Approximate size to download 139.6 MB
[OK!]
CPU times: user 36.6 ms, sys: 7.7 ms, total: 44.3 ms
Wall time: 3.61 s


In [ ]:
nlp_bert_five_hund=nlp_pipeline_Bert.fit(train)

In [ ]:
nlp_bert_five_hund.write().overwrite().save('gdrive/MyDrive/Colab_notebook/Models_Pipelines/bert_500_readmission')

In [ ]:
from pyspark.ml.pipeline import PipelineModel
nlp_bert_five_hund= PipelineModel.load("gdrive/MyDrive/Colab_notebook/Models_Pipelines/bert_500_readmission/")

In [ ]:
# transform the training and test set
processed_train=nlp_bert_five_hund.transform(train)
processed_test=nlp_bert_five_hund.transform(test)

In [ ]:
# Combine rows based on admission id
from pyspark.sql.functions import collect_list
from pyspark.sql import functions as F
processed_train_combined = processed_train.groupby('HADM_ID').agg(collect_list('features').alias("features"),F.min(processed_train.label))
processed_test_combined= processed_test.groupby('HADM_ID').agg(collect_list('features').alias("features"),F.min(processed_test.label))

In [ ]:
processed_test_combined.show()

+-------+--------------------+----------+
|HADM_ID|            features|min(label)|
+-------+--------------------+----------+
| 120327|[[0.0921542346477...|       1.0|
| 122377|[[0.0384613610804...|       1.0|
| 124703|[[0.1649432927370...|       1.0|
| 126716|[[0.0280453320592...|       1.0|
| 127229|[[0.1691382229328...|       1.0|
| 127525|[[0.1240143254399...|       1.0|
| 127964|[[0.0712104514241...|       0.0|
| 137634|[[0.0280135795474...|       1.0|
| 168079|[[0.0622289851307...|       1.0|
| 168808|[[0.0853902623057...|       1.0|
| 101738|[[0.2376184910535...|       0.0|
| 101756|[[0.1153774484992...|       1.0|
| 104084|[[0.1021297946572...|       1.0|
| 108589|[[0.1012754812836...|       1.0|
| 115838|[[-0.069325789809...|       1.0|
| 117383|[[0.1129594817757...|       1.0|
| 131111|[[0.0992296636104...|       1.0|
| 150665|[[0.0606598705053...|       1.0|
| 182711|[[0.0030397975351...|       1.0|
| 188057|[[-0.027117162942...|       1.0|
+-------+--------------------+----

In [ ]:
# Save the transformed training and test set as orc
processed_train_combined.write.orc("gdrive/MyDrive/Colab_notebook/transformed_data/bert_500_train")
processed_test_combined.write.orc("gdrive/MyDrive/Colab_notebook/transformed_data/bert_500_test")

In [ ]:
pd_train=processed_train_combined.toPandas()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/pandas/conversion.py:87: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


In [ ]:
pd_test=processed_test_combined.toPandas()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/pandas/conversion.py:87: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


In [ ]:
pd.set_option('display.max_colwidth', None)
pd_test['features'][0]

[DenseVector([0.0922, -0.0392, 0.0624, -0.1581, 0.1529, -0.1255, 0.1055, -0.0642, 0.0801, -0.2005, 0.1021, -0.1915, -0.0902, -0.3402, 0.1127, -0.3634, -0.3042, 0.3004, 0.1564, 0.2389, 0.2338, -0.1066, -0.2129, 0.0182, -0.0363, 0.1554, -0.5223, 0.1434, 0.0287, 0.1994, -0.1604, -0.0372, -0.1737, 0.1268, -0.7049, 0.2984, -0.3897, -0.0976, 0.2272, 0.0617, -0.2109, 0.0722, -0.0901, -0.2205, -0.0, -0.4101, -0.1756, 0.1586, 0.3681, -0.4862, -0.1079, 0.186, -0.0009, 0.1073, 0.1276, 0.0969, 0.2232, 0.1403, -0.2201, 0.037, 0.0517, 0.1135, -0.7833, 0.0378, -0.0355, -0.1085, -0.5104, -0.3516, 0.1113, 0.3572, 0.1642, -0.297, -0.2195, -0.2544, 0.664, -0.264, -0.2465, -0.1743, -0.2322, -0.3486, -0.0208, -0.0262, -0.4049, 0.2748, -0.1267, 0.3098, 0.4756, 0.1902, 0.0268, -0.2775, -0.0969, -0.0863, 0.2576, -0.544, 0.0946, 0.2653, 0.0269, 0.1624, 0.3014, -0.3021, -0.0438, -0.3842, -0.1972, -0.3052, -0.2254, -0.4373, 0.5011, -0.0269, 0.0452, -0.1322, 0.2166, -0.2457, 0.0457, 0.2552, 0.3509, 0.0249, 0.1615

In [ ]:
pd_test['features'][10][0]

0.23761849105358124

In [ ]:
# Define function to compute average embedding vector
def average_emb(df):
  for i in range(len(df)):
    new_embedding_list=[]
    embedding_list=df['features'][i]
    for k in range(len(embedding_list)):
      sentence_embedding=embedding_list[k]
      new_embedding_list.append(sentence_embedding)
    df['features'][i]=[sum(sub_list) / len(sub_list) for sub_list in zip(*new_embedding_list)]
  return(df)

In [ ]:
pd_train=average_emb(pd_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
pd_train.to_csv('gdrive/MyDrive/Colab_notebook/transformed_data/bert_500_train.csv')

In [ ]:
pd_test=average_emb(pd_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
pd_test.to_csv('gdrive/MyDrive/Colab_notebook/transformed_data/bert_500_test.csv')

In [ ]:
pd_train=pd.read_csv('gdrive/MyDrive/Colab_notebook/transformed_data/bert_500_train.csv')
pd_test=pd.read_csv('gdrive/MyDrive/Colab_notebook/transformed_data/bert_500_test.csv')

In [ ]:
pd_train.label=pd_train['min(label)'].astype("int")
pd_test.label=pd_test['min(label)'].astype("int")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [ ]:
X_train=pd_train.features
y_train=pd_train['min(label)']

X_test=pd_test.features
y_test=pd_test['min(label)']

In [ ]:
train_invalid_index=[]
for i in range(len(X_train)):
  if X_train[i]=='[]':
    train_invalid_index.append(i)
X_train=X_train.drop(train_invalid_index)

In [ ]:
y_train=y_train.drop(train_invalid_index)

In [ ]:
test_invalid_index=[]
for i in range(len(X_test)):
  if X_test[i]=='[]':
    test_invalid_index.append(i)
X_test=X_test.drop(test_invalid_index)

In [ ]:
y_test=y_test.drop(test_invalid_index)

In [ ]:
X_train_trans=[]
for doc in X_train:
    embedding=doc[1:-1]
    embedding_list=embedding.split(",")
    str_to_num_list=[]
    for num_str in embedding_list:
        str_to_num_list.append(float(num_str))
    X_train_trans.append(str_to_num_list)

In [ ]:
X_test_trans=[]
for doc in X_test:
    embedding=doc[1:-1]
    embedding_list=embedding.split(",")
    str_to_num_list=[]
    for num_str in embedding_list:
        str_to_num_list.append(float(num_str))
    X_test_trans.append(str_to_num_list)

Modeling

In [ ]:
!pip install scikit-optimize

     |████████████████████████████████| 100 kB 1.1 MB/s 


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from skopt import BayesSearchCV
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
# bert vs Logistic regression
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]

param= dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
scoring='roc_auc'

logistic_clf_bert = BayesSearchCV(estimator=LogisticRegression(), search_spaces=param, scoring=scoring, n_jobs=-1, cv=cv)

In [ ]:
logistic_clf_bert.fit(X_train_trans,y_train)

In [ ]:
logistic_clf_bert_best=logistic_clf_bert.best_score_

In [ ]:
logistic_clf_bert_best

0.6474705748220536

In [ ]:
y_prob_logistic_clf_bert = logistic_clf_bert.predict_proba(X_test_trans)
roc_auc_y_prob_logistic_clf_bert=roc_auc_score(y_test, y_prob_logistic_clf_bert[:,1])

In [ ]:
roc_auc_y_prob_logistic_clf_bert

0.6467185243656803

In [ ]:
# Bert and Linear SVM 
c_values=[100, 10, 1.0, 0.1, 0.01]

param= dict(C=c_values)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
scoring='roc_auc'

lsvc_clf_bert = BayesSearchCV(estimator=LinearSVC(), search_spaces=param, scoring=scoring, n_jobs=-1, cv=cv)

In [ ]:
lsvc_clf_bert.fit(X_train_trans,y_train)

In [ ]:
lsvc_clf_bert_best=lsvc_clf_bert.best_score_

In [ ]:
lsvc_clf_bert_best

0.6466278427957945